# [GPT-Assign03] An LCEL Chain with a Memory
-----
### Lecture05: Memory


* Implement **an LCEL chain with a memory** that uses one of the memory classes we learned about.

* The chain should take the title of a movie and reply with three emojis that represent the movie. 
    * (i.e "Top Gun" -> "🛩️👨‍✈️🔥". "The Godfather" -> "👨‍👨‍👦🔫🍝 ")
    
    * input : the title of a movie 

    * output: three emojis that represent the movie 

* Provide examples to the chain using **< FewShotPromptTemplate >** or **< FewShotChatMessagePromptTemplate >** to make sure it always replies with three emojis.


* **Tip⭐️**) To check that the memory is working ask the chain about two movies and then in another cell ask the chain to tell you what is the movie you asked about first.



## Apply all the content learned so far..
------

* LC Expression Language

* FewShot: **FewShotChatMessagePromptTemplate()**

* Memory: **ConversationSummaryBufferMemory()**

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough 
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder 


llm = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

memory= ConversationSummaryBufferMemory( ## llm =/ chat
    llm =llm,
    max_token_limit=80,
    # memory_key="chat_history", 
    return_messages=True,
)


examples = [
    {
        "question": "The Godfather",
        "answer": "🛩️👨‍✈️🔥",
        },
        {
        "question": "Top Gun",
        "answer": "👨‍👨‍👦🔫🍝",
        },
        {
        "question": "Frozen",
        "answer": "❄️👸🏼⛄",
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human","{question}?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt= example_prompt,
    examples= examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ('system', "You are a movie enthusiast. \
     Therefore, when given a movie title, you can output the three emojis that best represent the content of that movie.\
     If it's not a movie title, you don't respond with emojis but just answer the question that you are given"),
    example_prompt,
    MessagesPlaceholder(variable_name="history"),
    ("human","{question}?") 

])

def load_memeory(input):       
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memeory) | final_prompt | llm

def invoke_chain(question):     
    result= chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": result.content},)
    print(result)




In [2]:
invoke_chain( "Zootopia")

🐰🦊👮‍♂️content='🐰🦊👮\u200d♂️'


In [3]:
invoke_chain("Parasite")

👨‍👩‍👧‍👦🏠🔪content='👨\u200d👩\u200d👧\u200d👦🏠🔪'


In [4]:
invoke_chain("Little Forest: Summer/Autumn")

🌳🍲🌻The human mentions "Zootopia" and the AI responds with emojis of a rabbit, fox, and police officer.content='🌳🍲🌻'


In [5]:
invoke_chain("What was the last movie that a human asked about?")

The last movie title that was asked about was "Little Forest: Summer/Autumn".The human mentions "Zootopia" and the AI responds with emojis of a rabbit, fox, and police officer. When the human brings up "Parasite," the AI responds with emojis representing a family, a house, and a knife.content='The last movie title that was asked about was "Little Forest: Summer/Autumn".'


In [6]:
invoke_chain("What was the first movie you were asked about? You can answer with emojis.")

🛩️👨‍✈️🔥The human mentions "Zootopia" and the AI responds with emojis of a rabbit, fox, and police officer. When the human brings up "Parasite," the AI responds with emojis representing a family, a house, and a knife. The human then asks about "Little Forest: Summer/Autumn" and the AI responds with emojis of a tree, a bowl of soup, and sunflowers. The human inquires about the last movie a human asked about.content='🛩️👨\u200d✈️🔥'
